In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
clean_arr = np.loadtxt("wifi_db/clean_dataset.txt")
noisy_arr = np.loadtxt("wifi_db/noisy_dataset.txt")

In [35]:
LABEL_COL = 7
labels = [1, 2, 3, 4]

In [33]:
clean_arr[:, LABEL_COL]
ar = np.count_nonzero(clean_arr[:, [LABEL_COL]] == 1)
ar

500

In [36]:
def calc_num_of_rows_of_each_label(dataset, label):
    return np.count_nonzero(clean_arr[:, [LABEL_COL]] == label)
    

In [37]:
def entropy(dataset):
    dataset_size = dataset.len()
    res = numpy.zeros(labels.len())
    for i in range(len(labels)):
        res[i] = calc_num_of_rows_of_each_label(dataset, label[i]))
    res = res / dataset_size
    log_res = np.log2(res)
    res = res * log_res
    return - np.sum(res)  

In [38]:
def remainder(split, dataset):
    left_info_gain = (split + 1) / dataset.len() * entropy(dataset[:split + 1])
    right_info_gain = (dataset.len() - split) * entropy(dataset[split + 1 :])
    return left_info_gain + right_info_gain

def info_gain(col, i, split):
    return entropy(col) - remainder(i, col.len(), col)

In [40]:
def find_split(trng_data):
    return

In [41]:
def decision_tree_learning(training_dataset, depth):
    return